In [5]:
import json
import requests
import json_delta


'''
------------------------------------------------
---  A note about comparison techniques used ---
------------------------------------------------
json_delta is best for serializing/deserializing structures and
minimizing comm overhead.  It's may not be ideal for specialized
comparison of existing JSON
'''

1


"\n------------------------------------------------\n---  A note about comparison techniques used ---\n------------------------------------------------\njson_delta is best for serializing/deserializing structures and\nminimizing comm overhead.  It's may not be ideal for specialized\ncomparison of existing JSON\n"

In [21]:
import os
import glob   # Wildcard search

file_pattern = "snapshots/"
file_pattern += "HealthData.gov[_][0-9][0-9][0-9][0-9][-][0-9][0-9][-][0-9][0-9][_]data.json"

#print "glob.glob("+file_pattern+")"
#print glob.glob(file_pattern)


In [35]:
def load_file(json_file_name):
    with open(json_file_name) as json_file:
        json_data_struct = json.load(json_file)
        return json_data_struct

file_pattern = "snapshots/"
file_pattern += "HealthData.gov[_][0-9][0-9][0-9][0-9][-][0-9][0-9][-][0-9][0-9][_]data.json"
file_list = glob.glob(file_pattern)


json_data_list = [dict() for x in range(len(file_list))]


'''
#--- Test comparison ---
json_data_list[0] = load_file(file_list[0])
json_data_list[1] = load_file(file_list[1])
totals = compare_datasets(json_data_list[0],json_data_list[1])
print totals
'''

<generator object generate_udiff_lines at 0x1087d62d0>
 {
 "format": "text/csv",
 ...,
+ "_is_federal_dataset": false,
 
+ "keyword": [
+  "health"
+ ],
 
 "accessURL":
-  "https://data.kcmo.org/api/views/ks2s-yguy/rows.csv?accessType=DOWNLOAD",
+  "https://data.mo.gov/api/views/ks2s-yguy/rows.csv?accessType=DOWNLOAD",
 
 "webService":
-  "https://data.kcmo.org/resource/ks2s-yguy",
+  "https://data.mo.gov/resource/ks2s-yguy",
 
 "distribution":
-  [
-   {
-    "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.csv?accessType=DOWNLOAD", 
-    "identifier": "fb610ec4-d39e-497e-8a07-ca4e68a312bd", 
-    "format": "text/csv"
-   }, 
-   {
-    "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.json?accessType=DOWNLOAD", 
-    "identifier": "69034009-efed-4ee2-bef3-65822b6bd9d4", 
-    "format": "application/json"
-   }, 
-   {
-    "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.xml?accessType=DOWNLOAD", 
-    "identifier": "580bcfa2-f082-422d-abb0-57e08a31

In [278]:
# json_delta.load_and_diff('{"foo":"bar"}', '{"foo":"baz"}', verbose=False)
# print "\n\n----------------------------\n\n"
test = json_delta.load_and_udiff('{"foo":"bar"}', '{"foo":"baz"}')
print test
print '\n'.join(test)

<generator object generate_udiff_lines at 0x107b1f0f0>
 {
 "foo":
-  "bar"
+  "baz"
 }


In [269]:
test = json_delta.udiff(left='{"foo":"bar"}', right='{"foo":"baz"}')
print test
print '\n'.join(test)

<generator object generate_udiff_lines at 0x10b069d70>
-"{\"foo\":\"bar\"}"
+"{\"foo\":\"baz\"}"


In [29]:
# recursively sort any lists it finds (and convert dictionaries
# to lists of (key, value) pairs so that they're orderable):

def ordered_json(obj):
    if isinstance(obj, dict):
        return sorted((k, ordered_json(v)) for k, v in obj.items())
    if isinstance(obj, list):
        return sorted(ordered_json(x) for x in obj)
    else:
        return obj
    
# print              json_data[1]['dataset'][0]
#print
# print ordered_json(json_data[1]['dataset'][0])

In [21]:
# recursively sort any lists it finds (and convert dictionaries
# to lists of (key, value) pairs so that they're orderable):
import collections

def ordered_json2(obj,max_depth):

    if max_depth == 1: return obj + "(max depth reached)"
    
    if isinstance(obj, dict):
        print "Instance: dictionary -- Type " + str(type(obj))
        return sorted((k, ordered_json2(v, max_depth-1)) for k, v in obj.items())
        #return collections.OrderedDict(sorted(obj.items()))
        ## return sorted(ordered_json2({k:v}, max_depth-1) for k, v in obj.items())
    if isinstance(obj, list):
        print "Instance: list -- Type " + str(type(obj))
        return sorted(ordered_json2(x,max_depth-1) for x in obj)
    else:
        return obj

test = {}
#print ordered_json2(,5)
#print ordered_json2({'def1':['abc2','abc1',{'lab1':"c",'lab0':"b"}],'def0':'xyz'},5)
before_test = {'def1':['abc2','abc1',{'lab1':"a",'lab0':"b"}],'def0':'xyz'}
after_test  = {'def1':['abc2','abc1',{'lab1':"a",'lab0':"c"}],'def0':'xyz'}
print ordered_json2([before_test,after_test],5)

#print              json_data[1]['dataset'][0]
print
#print ordered_json(json_data[1]['dataset'][0])
'''
Convert tuple to dictionary: But this isn't sufficient

'''

Instance: list -- Type <type 'list'>
Instance: dictionary -- Type <type 'dict'>
Instance: list -- Type <type 'list'>
Instance: dictionary -- Type <type 'dict'>
Instance: dictionary -- Type <type 'dict'>
Instance: list -- Type <type 'list'>
Instance: dictionary -- Type <type 'dict'>
[[('def0', 'xyz'), ('def1', [[('lab0', 'b(max depth reached)'), ('lab1', 'a(max depth reached)')], 'abc1', 'abc2'])], [('def0', 'xyz'), ('def1', [[('lab0', 'c(max depth reached)'), ('lab1', 'a(max depth reached)')], 'abc1', 'abc2'])]]



"\nConvert tuple to dictionary: But this isn't sufficient\n\n"

In [218]:
len(json_data[0]['dataset'])

1633

In [1]:
# Create a dictionary of values for comparison
import json_delta

def compare_datasets(dataset_list_before, dataset_list_after):
    json_compare_dict = {}
    dataset_list_diff = {"Counts":{"Added":0, "Deleted":0, "Changed":0, "No Change":0},
                         "Diff":""}

    '''    
    #=== Sort entire datasets recursively for easier comparison later ===
    print("\n\n===dataset_before===")
    print(dataset_before)
    print("\n\n===dataset_after===")
    print(dataset_after)
    #dataset_before = ordered_json(dataset_before)
    #dataset_after  = ordered_json(dataset_after )
    print("\n\n===ordered_json(dataset_before===")
    print(dataset_before)
    print("\n\n===ordered_json(dataset_after===")
    print(dataset_after)
   
    '''

    #=== First load the "after" values ===
    for index, dataset_after in enumerate(dataset_list_after):

        check_key = dataset_after['identifier']

        json_compare_dict[check_key] = {'Status'    :"Added",
                                        'Before'    :None,
                                        'After'     :dataset_after,
                                        'Difference':None
                                       }
        dataset_list_diff["Counts"]["Added"] += 1


    #=== Second load the "before" values ===
    for index, dataset_before in enumerate(dataset_list_before):

        check_key = dataset_before['identifier']

        if check_key in json_compare_dict:
            # Not deleted, so check for differences
            
            dataset_after = json_compare_dict[check_key]['After']

            # Must compare sorted versions of json struct
            if ordered_json(dataset_after) == ordered_json(dataset_before):
                diff_status = "No Change"
            else:
                diff_status = "Changed"
           
                # Analyze difference only if changed
                udiff_list = json_delta.udiff(
                    dataset_before, 
                    dataset_after
                     )
                udiff_output = '\n'.join(udiff_list)
                dataset_list_diff["Diff"] = udiff_output
                
                
                print "\n\n==dumps_before==========\n\n"
                print 'type='+str(type(dataset_before))
                print json.dumps(dataset_before)
                print "\n\n==dumps_after==========\n\n"
                print 'type='+str(type(dataset_after))
                print json.dumps(dataset_after)
                print "\n\n==load udiff list==========\n\n"
                print 'type='+str(type(udiff_list))
                print udiff_list
                print "\n\n==load udiff output==========\n\n"
                print 'type='+str(type(udiff_output))
                print udiff_output

                
                return dataset_list_diff

        else:
            # Deleted
            diff_status = "Deleted"
            
            
        json_compare_dict[check_key] = {'Status':diff_status,
                                        'Before':dataset_before
                                        }
        dataset_list_diff["Counts"][diff_status] += 1
        dataset_list_diff["Counts"]["Added"    ] -= 1
            
    dataset_list_diff["Counts"]["Added"] = max(0,dataset_list_diff["Counts"]["Added"])
    return dataset_list_diff
    
    
    
#--- Test it ---
#totals = compare_datasets(json_data[0]['dataset'][:2],json_data[1]['dataset'][:2])
#totals = compare_datasets(json_data[0]['dataset'],json_data[1]['dataset'])
#print totals
totals = compare_datasets(json_data_list[0],json_data_list[1])
print totals["Counts"]
print totals["Diff"]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-a02407370c03>, line 62)

In [57]:
if "landingPage" in json_data_list[0][0]: print json_data_list[0][0]["landingPage"]
if "publisher" in json_data_list[0][0]: print json_data_list[0][0]["publisher"]
if "bureauCode" in json_data_list[0][0]: print json_data_list[0][0]["bureauCode"]
print json.dumps(json_data_list[0][0], sort_keys=False, indent=4)


https://data.kcmo.org/resource/ks2s-yguy
State of Missouri
{
    "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.csv?accessType=DOWNLOAD", 
    "publisher": "State of Missouri", 
    "accessLevel": "Public", 
    "description": "Cooling Centers Sites in Missouri", 
    "title": "Missouri Cooling Centers Sites", 
    "format": "text/csv", 
    "landingPage": "https://data.kcmo.org/resource/ks2s-yguy", 
    "modified": "2013-03-19", 
    "theme": [
        "Health"
    ], 
    "dataQuality": true, 
    "contactPoint": "Health Data Initiative", 
    "identifier": "a2bded07-837d-4671-89be-748deb455f36", 
    "mbox": "Healthdata@hhs.gov", 
    "distribution": [
        {
            "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.csv?accessType=DOWNLOAD", 
            "identifier": "fb610ec4-d39e-497e-8a07-ca4e68a312bd", 
            "format": "text/csv"
        }, 
        {
            "accessURL": "https://data.kcmo.org/api/views/ks2s-yguy/rows.json?accessType=D

In [211]:
#=== Check status counts between consecutive days ===

totals = {}
for key, value in json_compare_dict.iteritems():
    #print dataset['identifier']
    check_status = value['Status']
    if check_status in totals:
        totals[check_status] += 1
    else:
        totals[check_status]  = 1
    #break

totals

{'Added': 33, 'Not New': 1633}

In [244]:
#json_data[0]['dataset'][0,1,3]
for dataset in json_data[0]['dataset'][:2]:
#    print dataset
    check_key = dataset['identifier']
#json_data[0]['dataset'][:2]

In [3]:
import requests
# find list of files
directory_url = 'http://data-staging.civicagency.org/archive/datajson'
directory_response = requests.get(directory_url)



In [25]:
#directory_response.status_code
#directory_response.text


In [ ]:
# Get HTML page
from lxml import html
tree = html.fromstring(directory_response.content)
tree.xpath

In [29]:
from bs4 import BeautifulSoup
directory_soup = BeautifulSoup(directory_response.text)

In [124]:
suffix_url = '/49021.json'
datajson_url_list = []

for a_tag in directory_soup.find_all('a', href=True):
    a_text = a_tag.text.replace("/", "")
    if valid_date(a_text): 
        print(directory_url+"/"+a_text+suffix_url)
        datajson_url_list.append(directory_url+"/"+a_text+suffix_url)
    #print("Found the URL:"+ a_tag['href'] + "   Text: "+ a_tag.text, valid_date(a_tag.text))
    #print re.sub('[/]', '', a_tag.text)

    
# Sorts list to start with most recent
datajson_url_list.sort(reverse=True)  




http://data-staging.civicagency.org/archive/datajson/2014-02-24/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-03-27/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-04-14/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-04-25/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-04-27/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-05-04/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-05-11/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-05-18/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-05-25/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-06-01/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-06-03/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-06-08/49021.json
http://data-staging.civicagency.org/archive/datajson/2014-06-15/49021.json
http://data-staging.civic

In [137]:
read_limit = 10

for index,url in enumerate(datajson_url_list):
    print url
    head = requests.head(url,headers={'Accept-Encoding': 'identity'})
    # print (head.headers['content-length'])

    if head.status_code == 200:
        break 
    else:
        print head.status_code

    if read_limit:
        if index+1 >= read_limit:
            break    



http://data-staging.civicagency.org/archive/datajson/2015-12-11/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-10/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-09/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-08/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-07/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-06/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-05/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-04/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-03/49021.json
404
http://data-staging.civicagency.org/archive/datajson/2015-12-02/49021.json
404


In [158]:
# Function to obtain size of page from headers
def get_page_size(url):
    head = requests.head(url,headers={'Accept-Encoding': 'identity'})
    # print (head.headers['content-length'])

    if head.status_code == 200:
        if 'Content-Length' in head.headers:
            size = head.headers['Content-Length']
            return size
    return 0   # When no size is avail


#--- Test function ---
print datajson_url_list[100]+": "+str(get_page_size(datajson_url_list[100]))
print datajson_url_list[0]  +": "+str(get_page_size(datajson_url_list[0  ]))


http://data-staging.civicagency.org/archive/datajson/2015-09-02/49021.json: 1840306
http://data-staging.civicagency.org/archive/datajson/2015-12-11/49021.json: 0


In [70]:
# Function to help find relevant links 
import datetime
def valid_date(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        # raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        return False

print(valid_date('2003-12-223'))

False


In [70]:
Accept
requests.head(url,headers={'Accept-Encoding': 'identity'})

r = requests.head('http://pymotw.com/2/urllib/index.html',headers={'Accept-Encoding': 'identity'})
>>> r.headers['content-length']

/Applications/anaconda/anaconda/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [41]:

#json_data[0] #['Dataset']
json_data[0]['dataset'][0]

{u'@type': u'dcat:Dataset',
 u'accessLevel': u'public',
 u'bureauCode': [u'009:00'],
 u'contactPoint': {u'fn': u'Jackie Haven',
  u'hasEmail': u'mailto:noemailprovided@usa.gov'},
 u'description': u'<p>MyPyramid Food Data provides information on the total calories; calories from solid fats, added sugars, and alcohol (extras); MyPyramid food group and subgroup amounts; and saturated fat content of over 1,000 commonly eaten foods with corresponding commonly used portion amounts. This information is key to help consumers meet the recommendations of the Dietary Guidelines for Americans and manage their weight by understanding how many calories are consumed from "extras." CNPP has created an interactive tool from this data set available on the web at MyFood-a-pedia.gov. A mobile version is coming soon to provide consumers with assistance on-the-go.</p>\n',
 u'identifier': u'USDA-FNS-00001',
 u'keyword': [u'health'],
 u'language': [u'en'],
 u'modified': u'2015-08-29',
 u'programCode': [u'009: